### Load dataset

In [ ]:
import os
import wandb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from os.path import join, exists
from os import makedirs

In [ ]:
run_name = "verysmall_balanced_cyclical"
wandb.init(project="binary_search_optimization", name=run_name)

In [ ]:
DATASET_DIR = "./datasets/dataset_verysmall_balanced.pkl"
WEIGHTS_DIR = join("./save_weights", run_name)
if not exists(WEIGHTS_DIR):
    makedirs(WEIGHTS_DIR)

MAX_MONSTER_NUM = 1000
MONSTER_HPS_COL = ["monster_hp_" + str(num) for num in range(1, MAX_MONSTER_NUM + 1)]
FEATURES_COL = ["focus_damage", "aoe_damage", *MONSTER_HPS_COL]
TARGET_COL = ["attack_num"]

In [ ]:
dataset = pd.read_pickle(DATASET_DIR)

# Log data distribution
for col in ["focus_damage", "aoe_damage", "attack_num"]:
    plt.hist(dataset[col])
    wandb.log({col: plt})

### Train test split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
bins = np.linspace(dataset[TARGET_COL].to_numpy().min(), dataset[TARGET_COL].to_numpy().max(), 100, dtype=int)
Y_bin = np.digitize(dataset[TARGET_COL].to_numpy(), bins)

try:
    train_set, test_set = train_test_split(dataset, random_state=42, shuffle=True, stratify=Y_bin)
except:
    train_set, test_set = train_test_split(dataset, random_state=42, shuffle=True)

X_train, Y_train = train_set[FEATURES_COL].to_numpy(), train_set[TARGET_COL].to_numpy()
X_test, Y_test = test_set[FEATURES_COL].to_numpy(), test_set[TARGET_COL].to_numpy()

In [ ]:
len(X_train), len(X_test)

### Normalization

In [ ]:
import joblib
from sklearn.preprocessing import MinMaxScaler

In [ ]:
X_scaler = MinMaxScaler() if not exists(join(WEIGHTS_DIR, "X_scaler.pkl")) else joblib.load(join(WEIGHTS_DIR, "X_scaler.pkl"))
X_train_scaled = X_scaler.fit_transform(X_train.astype(np.float32))
X_test_scaled = X_scaler.transform(X_test.astype(np.float32))

Y_scaler = MinMaxScaler() if not exists(join(WEIGHTS_DIR, "Y_scaler.pkl")) else joblib.load(join(WEIGHTS_DIR, "Y_scaler.pkl"))
Y_train_scaled = Y_scaler.fit_transform(Y_train.astype(np.float32))
Y_test_scaled = Y_scaler.transform(Y_test.astype(np.float32))

In [ ]:
# Save parameters for scalers
joblib.dump(X_scaler, join(WEIGHTS_DIR, "X_scaler.pkl"))
joblib.dump(Y_scaler, join(WEIGHTS_DIR, "Y_scaler.pkl"))

### Model

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LeakyReLU
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.losses import MAE
from utilities import LearningRateFinder, Cosine

In [ ]:
wandb.config.network_depth = 1
wandb.config.network_width = 16
wandb.config.activation = "LeakyReLU"
wandb.config.optimizer = "Adam"
wandb.config.loss = "MAE"
wandb.config.stepsize = 20
wandb.config.epochs = 160
wandb.config.batch_size = 32
wandb.config.validation_split = 0.2

In [ ]:
class SequenceDense(Model):
    def __init__(self):
        super().__init__()
        self.hidden_layers = []
        for _ in range(wandb.config.network_depth):
            self.hidden_layers.append(Dense(wandb.config.network_width, activation=LeakyReLU()))
        self.output_layer = Dense(1, activation="relu")
        
    def call(self, inputs):
        output = inputs
        for layer in self.hidden_layers:
            output = layer(output)
        output = self.output_layer(output)
        
        return output

In [ ]:
model_name = "model_{}_{}".format(wandb.config.network_depth, wandb.config.network_width)

model = SequenceDense()
model.build(input_shape=X_train_scaled.shape)
model.compile(optimizer=Adam(learning_rate=0.0001), loss="mae")
try:
    model.load_weights(join("./save_weights", model_name))
except:
    model.save_weights(join("./save_weights", model_name))

lr_finder = LearningRateFinder(model)
lr_finder.find((X_train_scaled, Y_train_scaled), start_lr=1e-10, epochs=20)
lr_finder.plot()

In [ ]:
plt.plot(lr_finder.losses)
wandb.log({"lr_finder": plt})

In [ ]:
base_lr = lr_finder.lrs[15000]
max_lr = lr_finder.lrs[25000]
wandb.log({"learning_rate_base": base_lr})
wandb.log({"learning_rate_max": max_lr})
wandb.log({"learning_rate_base_index": 15000})
wandb.log({"learning_rate_max_index": 25000})

In [ ]:
def history_plot(history):
    loss = history.history["loss"]
    val_loss = history.history["val_loss"]
    plt.subplot(2, 1, 1)
    plt.title("loss")
    plt.plot(loss)
    plt.subplot(2, 1, 2)
    plt.title("val_loss")
    plt.plot(val_loss)
    
class CLR(tf.keras.callbacks.Callback):
    def __init__(self, *args, **kwargs):
        self.clr = Cosine(*args, **kwargs)
        self.history = []

    def on_epoch_end(self, epoch, logs={}):
        lr = self.clr()
        self.history.append(lr)
    
        return lr
    
lr_updater = CLR(max_lr, base_lr, wandb.config.stepsize)

model = SequenceDense()
model.build(input_shape=X_train_scaled.shape)
model.compile(optimizer=Adam(learning_rate=lr_finder.lrs[20000]), loss="mae")
model.load_weights(join("./save_weights", model_name))
history = model.fit(X_train_scaled, Y_train_scaled, 
                    epochs=wandb.config.epochs, 
                    batch_size=wandb.config.batch_size, 
                    validation_split=wandb.config.validation_split, 
                    verbose=1,
                    callbacks=[lr_updater])
model.save_weights(join(WEIGHTS_DIR, "model_trained", model_name))

history_plot(history)

In [ ]:
loss = history.history["loss"]
plt.title("train_loss")
plt.plot(loss)
wandb.log({"train_loss": plt})

val_loss = history.history["val_loss"]
plt.title("val_loss")
plt.plot(loss)
wandb.log({"val_loss": plt})

test_loss = model.evaluate(X_test_scaled, Y_test_scaled, verbose=0)
wandb.log({"test_loss": test_loss})

lr_history = lr_updater.history
plt.title("learning_rate_history")
plt.plot(lr_history)
wandb.log({"test_loss": plt})

In [ ]:
pred = model.predict(X_train_scaled)
loss = MAE(Y_train_scaled, pred)
plt.scatter(Y_train_scaled, loss)
wandb.log({"loss_dist": wandb.Image(plt, caption="loss_dist")})
print("Average loss: {}".format(np.mean(loss.numpy())))